In [3]:
import hublib
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import scipy
import numpy as np
from decimal import Decimal
import unicodedata
from unicodedata import lookup as GL
import sympy as sy
from joblib import Parallel, delayed
from lmfit import Model
import warnings
import zipfile
from zipfile import ZipFile
import os
# from hublib.ui import FileUpload, Download

In [4]:
### Import spectrum and do preliminary calculation here
### Output: I_raw(+ve), W


# import the file. We usually allow users to upload their own file
path = "./data/raw/SS_20190529_1_4_clean.txt"
data = pd.read_txt(path)

# calculate the number of rows and columns in the file. 
# We use this to determine the format of the file and import the data accordingly
cols = data.shape[1]
rows = data.shape[0]

# import data as a DataFrame, such that the first column has the frequency and the second column has the intensity
if cols == 1:
    data = pd.Dataframe(data.iloc[0:row/2, 0], data.iloc[rows/2:rows, 0])
elif cols == 2:
    if type(data.iloc[0,0]) is str:
        data = data.iloc[1:rows, :]
    else: 
        data = data
else: 
    data = []
    
# create the frequency and intensity numpy arrays
frequency = np.array(data.iloc[:,0])
intensity = np.array(data.iloc[:,1])
length = len(frequency)

# remove all negative values from the two arrays
a = 0
for i in range(length):
    if frequency[i]<=0:
        a = a + 1
    else:
        break
frequency = frequency[a:]
intensity = intensity[a:]

# normalise the intenisty
intensity_norm = []
for i in intensity:
    intensity_norm.append((i-np.min(intensity))/(np.max(intensity)-np.min(intensity)))

AttributeError: module 'pandas' has no attribute 'read_txt'

In [ ]:
### Fit BL
### Input: I_raw, W, *args (degree, here)
### Output: I, W

# define the baseline subtraction function
def fit_BL(degree, x, y):
    n = degree
    I_raw = np.array(y)
    W = np.array(x)
    
    # perform baseline fitting
    polyx = np.array([W[0],W[int(len(W)/2)],W[len(W)-1]])
    polyy = np.array([I_raw[0],I_raw[int(len(W)/2)],I_raw[len(W)-1]])        
    bkgfit = np.polyfit(W,I_raw,degree)
    bkgpoly = 0
    for i in range(n):
        bkgpoly = bkgpoly + (bkgfit[i]*W**(n-i))
    I_raw = I_raw-bkgpoly
    
    # Flatten the baseline
    m = (I_raw[len(W)-1]-I_raw[0])/(W[len(W)-1]-W[0])
    b = I_raw[len(W)-1]-m*W[len(W)-1]
    bkglin = m*W+b
    I_raw = I_raw-bkglin
    
    # re-normalise the baseline subracted spectrum
    I_BL = []
    I_BL = ((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)))
    
    return I_BL

In [ ]:
y = intensity_norm
window_length = 7
polyorder = 2

y_smooth = scipy.signal.savgol_filter(y, window_length, polyorder, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)

In [ ]:
### define parameters here. 
### Degree (so far, degree of baseline polynomial)

degree = 3

# call the baseline subtraction function
I_BL = fit_BL(degree, frequency, y_smooth)

In [ ]:
### Plot [I_raw vs W] & [I vs W]

fig, ax = plt.subplots(nrows=2,ncols=1, figsize=(8,8)) # create a figure with sub-plots
ax[0].plot(frequency, intensity_norm); # display the output image
ax[1].plot(frequency, I_BL) # display the original image

# label the images
ax[0].set_title('Normalised Spectrum')
ax[1].set_title('Baseline-subtracted Spectrum')